In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os, shutil
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import pandas as pd
import mdtraj as md
import pytraj as pt
import nglview as nv
import seaborn as sns

In [3]:
# Matplotlib options
%matplotlib inline
matplotlib.style.use('ggplot')
pylab.rcParams['figure.figsize'] = 12, 8

In [4]:
top='../centered_traj/CD44_PIC.pdb'
trajectory='../centered_traj/CD44_PIC_no_WAT.xtc'

In [5]:
traj=md.load_xtc(trajectory, top=top)
traj

KeyboardInterrupt: 

In [6]:
view=nv.show_mdtraj (traj)
view.clear_representations ()
view.add_ball_and_stick (selection='protein', aspectRatio=10)
view.add_ball_and_stick(selection='PIC', aspectRatio=10, color='purple')
view

NGLWidget(count=218001)

In [8]:
print(traj.topology.select('resid 1 to 150'))

[   8    9   10 ..., 1535 1536 1537]


In [ ]:
rmsd=md.rmsd (traj, reference=traj, frame=0)
plt.plot (rmsd)
plt.xlabel ('Frame')
plt.ylabel ('RMSD (Angstrom)')

In [ ]:
py_traj = pt.Trajectory(xyz=traj.xyz.astype('f8'), top=top)
py_traj

In [25]:
rmsd=pt.rmsd (py_traj,ref=0, mask=[':6-150'])
plt.plot (rmsd)
plt.xlabel ('Frame')
plt.ylabel ('RMSD (Angstrom)')

AttributeError: 'numpy.ndarray' object has no attribute 'top'

In [ ]:
rmsf = pt.rmsf(py_traj, mask='@CA')
plt.plot(rmsf.T[0], rmsf.T[1])
plt.xlabel('atom')
plt.ylabel('RMSF (Angstrom)')

# Hbonds

In [ ]:
hbonds_acceptor=pt.search_hbonds (py_traj,distance=3.0, series=True, options='acceptormask :PIC')
hbonds_donor=pt.search_hbonds (py_traj,distance=3.0, series=True, options='donormask :PIC')
print ('Acceptor pairs:')
print (hbonds_acceptor.donor_acceptor)
print ('Donor pairs:')
print (hbonds_donor.donor_acceptor)

## Acceptor

In [ ]:
distance_mask = hbonds_acceptor.get_amber_mask()[0]
print('hbond distance mask: {} \n '.format(distance_mask))
print ((len (distance_mask)),'hbond where found')
angle_mask = hbonds_acceptor.get_amber_mask()[1]
print('hbond angle mask: {} \n'.format(angle_mask))
print("hbond data")
print(hbonds_acceptor.data) # 1: have hbond; 0: does not have hbond

In [ ]:
# compute distance between donor-acceptor for ALL frames (also include frames that do not form hbond)
dist_acceptor = pt.distance(py_traj,hbonds_acceptor.get_amber_mask()[0])
print('all hbond distances: ', dist_acceptor)

In [ ]:
angle_acceptor = pt.angle(py_traj, hbonds_acceptor.get_amber_mask()[1])
angle_acceptor

In [ ]:
a=np.arange (len(distance_mask))
for i in range (len(a)):
plt.figure ()
plt.subplot(3,1,1)
plt.plot(hbonds_acceptor.data [i])
plt.xlabel (hbonds_acceptor.donor_acceptor [i])
plt.ylabel ('Frecuency')
save=pd.DataFrame ([hbonds_acceptor.data[i]])
t=save.transpose ()
#t.to_csv('/home/angel/Escritorio/'+hbonds_acceptor.donor_acceptor[i]+'.csv')

## Donor

In [ ]:
distance_mask = hbonds_donor.get_amber_mask()[0]
print('hbond distance mask: {} \n '.format(distance_mask))
angle_mask = hbonds_donor.get_amber_mask()[1]
print('hbond angle mask: {} \n'.format(angle_mask))
print("hbond data")
print(hbonds_donor.data) # 1: have hbond; 0: does not have hbond

In [ ]:
# compute distance between donor-acceptor for ALL frames (also include frames that do not form hbond)
dist_donor = pt.distance(py_traj,hbonds_donor.get_amber_mask()[0])
print('all hbond distances: ', dist_donor)

In [ ]:
angle_donor = pt.angle(py_traj, hbonds_donor.get_amber_mask()[1])
angle_donor

In [ ]:
d=np.arange (len(distance_mask))
for i in range (len(d)):
plt.figure ()
plt.subplot(3,1,1)
plt.plot(hbonds_donor.data [i])
plt.xlabel (hbonds_donor.donor_acceptor [i])
plt.ylabel ('Frecuency')
#save=pd.DataFrame ([hbonds_donor.data[i]])
#t=save.transpose ()
#t.to_csv('/home/angel/Escritorio/'+hbonds_donor.donor_acceptor[i]+'.csv')

## Pictilisib

In [ ]:
pic=py_traj [':151']
pic

In [ ]:
rmsd= pt.rmsd (pic,ref=0)
plt.plot (rmsd)
plt.xlabel ('Frame')
plt.ylabel ('RMSD (A)')

In [ ]:
rmsf=pt.rmsf (pic)
plt.plot(rmsf.T[0], rmsf.T[1])
plt.xlabel('atom')
plt.ylabel('RMSF (Angstrom)')